# Eva 02

## prepare

In [ ]:
from os import path

import torch

# from modelscope.models import Model
# from modelscope.pipelines import pipeline
# from modelscope.preprocessors import Preprocessor
# from modelscope.utils.constant import Tasks
from modelscope.hub.snapshot_download import snapshot_download

from detectron2.config import get_cfg, LazyConfig
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger

from detectron2.data import MetadataCatalog
from detectron2.engine.defaults import DefaultPredictor
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer

In [ ]:
base_url = r'/mnt/workspace'
ckpt_base_pth = path.join(base_url, 'model')
data_base_path = path.join(base_url, "data")

eva02_base_path = '/mnt/workspace/model/zacbi2023/eva02'
eva02_config_path = path.join(eva02_base_path, 'projects/ViTDet/configs/eva2_o365_to_coco/eva2_o365_to_coco_cascade_mask_rcnn_vitdet_l_8attn_1536_lrd0p8.py')
eva02_weights_path = path.join(eva02_base_path, 'checkpoints/eva02_L_coco_seg_sys_o365.pth') 

input_image_path = path.join(data_base_path, 'image/input/cat02.jpg')
output_dir = path.join(data_base_path, 'image/output/')

eva02_model_id = r'zacbi2023/eva02'
eva02_ckpt_path = path.join(ckpt_base_pth, eva02_model_id)

### download model

In [ ]:
if not path.exists(eva02_base_path):
    snapshot_download('zacbi2023/eva02', cache_dir = ckpt_base_pth, ignore_file_pattern='eva02_L_ade_seg_upernet_sz640.pth')

### config base

In [ ]:
config_file = eva02_config_path
custum_cfg = ['MODEL.RETINANET.SCORE_THRESH_TEST', 0.5,
              'MODEL.ROI_HEADS.SCORE_THRESH_TEST', 0.5,
              'MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH', 0.5,
              'DATASETS.TEST', [],
            'MODEL.WEIGHTS', eva02_weights_path]

### load via LazyConfig

In [ ]:
cfg = LazyConfig.load(config_file)
LazyConfig.apply_overrides(cfg, [f"{key}={value}" for key, value in zip(custum_cfg[::2], custum_cfg[1::2])])

### load via get_cfg()

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(config_file)
cfg.merge_from_list([f"{key}={value}" for key, value in zip(custum_cfg[::2], custum_cfg[1::2])])

## Infer

In [ ]:
from detectron2.config import instantiate
model = instantiate(cfg.model)
model.to('cuda')

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer
DetectionCheckpointer(model).load(eva02_weights_path)
model.eval()

In [ ]:
original_image = read_image(input_image_path, format="BGR")
height, width = original_image.shape[:2]
image = torch.as_tensor(original_image.astype("float32").transpose(2, 0, 1))
inputs = {"image": image, "height": height, "width": width}
predictions = model([inputs])[0]

In [ ]:
original_image = original_image[:, :, ::-1]

In [ ]:
# 可视化器
visualizer = Visualizer(original_image, None,
                                instance_mode=ColorMode.IMAGE)

In [ ]:
cpu_device = torch.device("cpu")
instances = predictions["instances"].to(cpu_device)

In [ ]:
# 可视化输出
vis_output = visualizer.draw_instance_predictions(
                    predictions=instances)

In [ ]:
vis_output.save(path.join(output_dir, 'cat02.jpg'))